In [ ]:
import os
import shutil
import zipfile
from tqdm import tqdm

### 원본 데이터셋 압축해제

AI Hub에서 다운로드 받은 주행 중 건물 이미지 데이터셋을 원본으로 한다.

In [ ]:
cate_list = ["Training", "Validation"]
for cate in cate_list:
    src_dir = "E:/pgu_traffic_dataset/org/{}".format(cate)
    dst_dir = "E:/pgu_traffic_dataset/unzipped/{}".format(cate)
    file_list = os.listdir(src_dir)
    for file in tqdm(file_list):
        zipfile.ZipFile("{}/{}".format(src_dir, file)).extractall(dst_dir)

### 라벨 파일이 없는 이미지 파일 확인

In [ ]:
image_file_list = []
annotation_file_list = []
for cate in ["Training", "Validation"]:
    dst_dir = "E:/pgu_traffic_dataset/unzipped/{}".format(cate)
    
    image_file_list.extend([x.split(".")[0] for x in os.listdir("{}/{}".format(dst_dir, "images"))])
    annotation_file_list.extend([x.replace("_BBOX", "").split(".")[0] for x in os.listdir("{}/{}".format(dst_dir, "annotations")) if "BBOX" in x])

image_file_list.sort()
annotation_file_list.sort()
annotation_not_exist = []
for file in tqdm(image_file_list):
    if file not in annotation_file_list:
        annotation_not_exist.append(file)
len(annotation_not_exist) # -> 3,444개의 이미지 파일에 라벨 파일이 없음

### 라벨 파일이 있는 이미지만 데이터셋으로 분류하고 복사

In [ ]:
src_dir_training = "E:/pgu_traffic_dataset/unzipped/Training"
src_dir_validation = "E:/pgu_traffic_dataset/unzipped/Validation"
dst_dir_images = "E:/pgu_traffic_dataset/cleaned/images"
dst_dir_annotations = "E:/pgu_traffic_dataset/cleaned/annotations"
for file in tqdm(image_file_list):
    if file in annotation_file_list:
        img_src = None
        ann_src = None
        img_file_exist_t = os.path.isfile("{}/images/{}.png".format(src_dir_training, file))
        img_file_exist_v = os.path.isfile("{}/images/{}.png".format(src_dir_validation, file))
        ann_file_exist_t = os.path.isfile("{}/annotations/{}_BBOX.json".format(src_dir_training, file))
        ann_file_exist_v = os.path.isfile("{}/annotations/{}_BBOX.json".format(src_dir_validation, file))
        if img_file_exist_t is True:
            img_src = "{}/images/{}.png".format(src_dir_training, file)
        if img_file_exist_v is True:
            img_src = "{}/images/{}.png".format(src_dir_validation, file)
        if ann_file_exist_t is True:
            ann_src = "{}/annotations/{}_BBOX.json".format(src_dir_training, file)
        if ann_file_exist_v is True:
            ann_src = "{}/annotations/{}_BBOX.json".format(src_dir_validation, file)

        if img_src is not None or ann_src is not None:
            shutil.copy(img_src, "{}/{}.png".format(dst_dir_images, file))
            shutil.copy(ann_src, "{}/{}.json".format(dst_dir_annotations, file))

### 파일이 제대로 있는지 최종 체크

In [ ]:
dst_dir_images = "E:/pgu_traffic_dataset/cleaned/images"
dst_dir_annotations = "E:/pgu_traffic_dataset/cleaned/annotations"

img_file_list = [x.split(".")[0] for x in os.listdir(dst_dir_images)]
ann_file_list = [x.split(".")[0] for x in os.listdir(dst_dir_annotations)]

len(img_file_list), len(ann_file_list)

In [ ]:
for img_file in tqdm(img_file_list):
    if img_file not in ann_file_list:
        print(img_file)